## LSTM_decoder_no_map

In [ ]:
# common imports
import os
import math
import time
import random
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from typing import Dict
from tqdm import tqdm
from pathlib import Path
from tempfile import gettempdir

# interactive plot libraries
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot # download_plotlyjs, plot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)

# torch imports
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
import torch.nn.functional as F

#from torch_lr_finder import LRFinder

# l5kit imports
import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory

In [ ]:
#!rm -rf /kaggle/working/*

In [ ]:
print(torch.__version__)

In [ ]:
print(l5kit.__version__)

In [ ]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(total_trainable_params)
    return total_trainable_params

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

## Configs

In [ ]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': "../input/lyft-motion-prediction-autonomous-vehicles/",
    'model_params': {
        'model_architecture': '',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "LSTM_decoder_no_map",
        'lr': 5e-4,
        'weight_path': "../input/lyftlstm-decoder-no-maps/LSTM_decoder_no_map_1024k.pth",
        'lr_find' : False, 
        'train': True,
        'validate': False,
        'test': False
    },

    'raster_params': {
        'raster_size': [330, 180],
        'pixel_size': [0.4, 0.4],
        'ego_center': [0.25, 0.5],
        'map_type': 'stub_debug',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },
    
    'val_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },

    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'train_params': {
        'train_start_batch_index' : 32001,
        'max_num_steps': 32002,
        'checkpoint_every_n_steps': 2000,
        'reduction_factor' : 0.9,
        'step_size' : 1e5
    }
}

In [ ]:
NUMBER_OF_HISTORY_FRAMES = cfg['model_params']['history_num_frames'] + 1
RASTER_IMG_SIZE = cfg['raster_params']['raster_size'][0]
NUM_MODES = 3
NUMBER_OF_FUTURE_FRAMES = cfg['model_params']['future_num_frames']

### TRAIN FROM WHERE LEFT OFF, CHANGE THE STARTING INDICES VARIABLE ACCORDINGLY
TRAIN_BATCH_SIZE = cfg['train_data_loader']['batch_size'] 
TRAIN_START_BATCH_INDICES = cfg['train_params']['train_start_batch_index']
EXTENT_RANGE = 5.0 
MIN_FRAMES_FUTURE = 10

In [ ]:
EXTENT_RANGE = 5.0 
MAX_VELOCITY = 20.0
MAX_ACCELERATION = 2.0
MAX_YAW_RATE = np.deg2rad(45)
dt =cfg['model_params']['history_delta_time']

## Rasterize and initialise

In [ ]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
rasterizer = build_rasterizer(cfg, dm)

## Loss function

In [ ]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Model

In [ ]:
class LSTM_Decoder(nn.Module):
    def __init__(self, cfg: Dict, num_modes = 3):
        super(LSTM_Decoder, self).__init__()
        self.num_history_frames = cfg["model_params"]["history_num_frames"] + 1
        self.num_future_frames = cfg["model_params"]["future_num_frames"]
        self.num_modes = num_modes
        
        # preprocessing layers for inputs
        self.his_pos_fc   = nn.Linear(2 * self.num_history_frames, 32)
        self.his_avail_fc = nn.Linear(self.num_history_frames, 16)
        self.extend_fc    = nn.Linear(3, 3)
        
        # LSTM layers   
        self.embed_size  = 3 * (self.num_modes) # 2 positions and 1 confidence for each mode
        self.hidden_size = 32 + 16 + 3
        self.decoder     = nn.LSTMCell(self.embed_size, self.hidden_size, bias=True)
        self.lstm_tgt_fc = nn.Linear(self.hidden_size, self.embed_size)
        
    def prepare_data(self, data, device):
        target_availabilities = data["target_availabilities"].to(device)
        targets = data["target_positions"].to(device)
        bs = targets.shape[0]

        extent = (data['extent'].to(device) / EXTENT_RANGE).view(bs, -1)
        history_positions = data['history_positions'].to(device).view(bs, -1)
        history_availabilities = data['history_availabilities'].to(device).view(bs, -1)
        return [bs, 
                history_positions, history_availabilities, extent,
                targets, target_availabilities]
    
    def LSTM_decoder_step(self, bs, his_pos, his_avail, bb, tgts, tgt_avail, criterion):
        # preallocating memory for output
        LSTM_output = torch.zeros(bs, self.num_future_frames, self.embed_size).to(device)
        # first decoder input
        initial_input = torch.zeros(bs, self.embed_size).to(device) 
        loss = 0.0
        
        # initial hidden state
        in1 = model.his_pos_fc(his_pos)
        in2 = model.his_avail_fc(his_avail)
        in3 = model.extend_fc(bb)
        decoder_hidden = torch.cat((in1, in2, in3), dim=1)
        decoder_cellState = torch.zeros_like(decoder_hidden)
        
        # Decode hidden state in future trajectory
        decoder_input = initial_input        
        for t in range(self.num_future_frames):
            decoder_hidden, decoder_cellState = self.decoder(decoder_input, (decoder_hidden, decoder_cellState))
            decoder_output = self.lstm_tgt_fc(decoder_hidden)
            LSTM_output[:, t, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        pred, confidences = torch.split(LSTM_output, 2 * self.num_modes, dim=2)
        pred = pred.reshape(bs, self.num_modes, self.num_future_frames, 2)
        confidences = torch.softmax(torch.mean(confidences, dim =1), dim=1)
        loss = criterion(tgts, pred, confidences,tgt_avail)
        return loss, pred, confidences
    
    # create masks for history data 
    def mask_history_data(self, device, history_positions, history_availabilities):
        # mask for history availabbilities and history_positions for reducing overfit
        avail_mask = torch.rand(history_availabilities.shape).to(device) > 0.9
        pos_mask = torch.repeat_interleave(avail_mask, 2, dim=1).to(device)
        history_availabilities = history_availabilities.masked_fill(avail_mask, 0)
        history_positions = history_positions.masked_fill(pos_mask, 0)
        return history_positions, history_availabilities
    
    # inference / prediction pipeline
    def forward(self, data, device, criterion = pytorch_neg_multi_log_likelihood_batch):
        bs, his_pos, his_avail, bb, tgts, tgt_avail = self.prepare_data(data, device)
        return self.LSTM_decoder_step(bs, his_pos, his_avail, bb, tgts, tgt_avail, criterion)
    
    # Training forward function with addtional masks for history position and availability data
    def train_forward(self, data, device, criterion = pytorch_neg_multi_log_likelihood_batch):
        bs, his_pos, his_avail, bb, tgts, tgt_avail = self.prepare_data(data, device)
        his_pos, his_avail = self.mask_history_data(device, his_pos, his_avail)
        #print(his_pos[0])
        return self.LSTM_decoder_step(bs, his_pos, his_avail, bb, tgts, tgt_avail, criterion)

In [ ]:
# ==== INIT MODEL=================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device {device}')

In [ ]:
model = LSTM_Decoder(cfg)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=cfg["model_params"]["lr"])
criterion = pytorch_neg_multi_log_likelihood_batch
print(f'device {device}')

In [ ]:
#load weight if there is a pretrained model
weight_path = cfg["model_params"]["weight_path"]
if weight_path != "":
    checkpoint = torch.load(weight_path)

In [ ]:
#if weight_path != "":
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
#if weight_path != "":
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
print(find_no_of_trainable_params(model))

In [ ]:
print('Before lr : ' , optimizer.param_groups[0]['lr'])
optimizer.param_groups[0]['lr'] = cfg["model_params"]["lr"]
print('After lr  : ' , optimizer.param_groups[0]['lr'])

In [ ]:
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr= cfg["model_params"]["lr"],
                                          total_steps=cfg['train_params']['max_num_steps'],
                                          pct_start=0.4, div_factor=10)

## Train Dataloader

In [ ]:
# ===== INIT TRAIN DATASET============================================================
if (cfg['model_params']['train'] == True) or (cfg['model_params']['lr_find'] == True):
    train_cfg = cfg["train_data_loader"]
    train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer, 
                                 min_frame_history = cfg['model_params']['history_num_frames'],
                                 min_frame_future=MIN_FRAMES_FUTURE)
    
    print('Length of Train dataset is ' ,len(train_dataset))
    #print("==================================TRAIN DATA==================================")
    #print(train_dataset)
    
    sampled_indices = np.random.choice(len(train_dataset), size = len(train_dataset), replace = False)
    #print('Before slicing, start indices are ', sampled_indices[0:5])
    #print('TRAIN_START_INDICES', TRAIN_START_BATCH_INDICES * TRAIN_BATCH_SIZE)
    
    sampled_indices = sampled_indices[TRAIN_START_BATCH_INDICES * TRAIN_BATCH_SIZE:]
    #print('After slicing, start indices are ', sampled_indices[0:5])
    
    Datasampler = SubsetRandomSampler(sampled_indices)
    train_dataloader = DataLoader(train_dataset, sampler=Datasampler, batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
    print('Length of train_dataloader is ' ,len(train_dataloader))

## Validation Dataloader

In [ ]:
eval_base_path = '../input/lyft-validation-chopped-100/validate_chopped_100'
eval_cfg = cfg["val_data_loader"]
eval_zarr_path = str(Path(eval_base_path) / Path(dm.require(eval_cfg["key"])).name)
eval_mask_path = str(Path(eval_base_path) / "mask.npz")
eval_gt_path = str(Path(eval_base_path) / "gt.csv")

eval_zarr = ChunkedDataset(eval_zarr_path).open()
eval_mask = np.load(eval_mask_path)["arr_0"]
# ===== INIT DATASET AND LOAD MASK
eval_dataset = AgentDataset(cfg, eval_zarr, rasterizer, agents_mask=eval_mask)
eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                             num_workers=eval_cfg["num_workers"])
print(eval_dataset)

In [ ]:
def model_validation_score(model, pred_path):
    # ==== EVAL LOOP
    model.eval()
    torch.set_grad_enabled(False)

    # store information for evaluation
    future_coords_offsets_pd = []
    timestamps = []
    confidences_list = []
    agent_ids = []
    progress_bar = tqdm(eval_dataloader)

    for data in progress_bar:

        _, preds, confidences = model.forward(data, device)

        #fix for the new environment
        preds = preds.cpu().numpy()
        world_from_agents = data["world_from_agent"].numpy()
        centroids = data["centroid"].numpy()
        coords_offset = []

        # convert into world coordinates and compute offsets
        for idx in range(len(preds)):
            for mode in range(3):
                preds[idx, mode, :, :] = transform_points(preds[idx, mode, :, :], world_from_agents[idx]) - centroids[idx][:2]

        future_coords_offsets_pd.append(preds.copy())
        confidences_list.append(confidences.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())  
    
    write_pred_csv(pred_path,
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
               confs=np.concatenate(confidences_list),
              )
    
    metrics = compute_metrics_csv(eval_gt_path, pred_path, [neg_multi_log_likelihood, time_displace])
    for metric_name, metric_mean in metrics.items():
        print(metric_name, metric_mean)
    
    #return [future_coords_offsets_pd, confidences_list, timestamps, agent_ids]

## LR_finder 

In [ ]:
def plot_lr_finder_results(lr_finder): 
    # Create subplot grid
    fig = make_subplots(rows=1, cols=2)
    # layout ={'title': 'Lr_finder_result'}
    
    # Create a line (trace) for the lr vs loss, gradient of loss
    trace0 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['smooth_loss'],name='log_lr vs smooth_loss')
    trace1 = go.Scatter(x=lr_finder['log_lr'], y=lr_finder['grad_loss'],name='log_lr vs loss gradient')

    # Add subplot trace & assign to each grid
    fig.add_trace(trace0, row=1, col=1)
    fig.add_trace(trace1, row=1, col=2)
    iplot(fig, show_link=False)
    #fig.write_html(cfg['model_params']['weight_path'] + '_lr_find.html')

def find_lr(data_loader, init_value = 1e-8, final_value=1.0, beta = 0.98, num_batches = 200):
    assert(num_batches > 0)
    mult = (final_value / init_value) ** (1/num_batches)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    batch_num = 0
    avg_loss = 0.0
    best_loss = 0.0
    smooth_losses = []
    raw_losses = []
    log_lrs = []
    dataloader_it = iter(data_loader)
    progress_bar = tqdm(range(num_batches))
    
    for idx in progress_bar:
        batch_num += 1
        try:
            data = next(dataloader_it)
        except StopIteration:
            dataloader_it = iter(data_loader)
            data = next(dataloader_it)
        
        # Forward pass
        model.train()
        torch.set_grad_enabled(True)
        
        # handle exception in criterion
        try:
            loss, _, _ = model.train_forward(data, device)
        except:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) *loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 50 * best_loss:
            if len(smooth_losses) > 1:
                grad_loss = np.gradient(smooth_losses)
            else:
                grad_loss = 0.0
            lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                                 'smooth_loss':smooth_losses, 'grad_loss': grad_loss}
            return lr_finder_results
        
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
        
        #Store the values
        raw_losses.append(loss.item())
        smooth_losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # print info
        progress_bar.set_description(f"loss: {loss.item()},smoothed_loss: {smoothed_loss},lr : {lr}")

        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
    
    grad_loss = np.gradient(lr_finder_results['smooth_loss'])
    lr_finder_results = {'log_lr':log_lrs, 'raw_loss':raw_losses, 
                         'smooth_loss':smooth_losses, '': grad_loss}
    return lr_finder_results

In [ ]:
if cfg['model_params']['lr_find'] == True:
    lr_finder_results = find_lr(train_dataloader)
    plot_lr_finder_results(lr_finder_results)

## Training loop

In [ ]:
# ==== TRAINING LOOP =========================================================
if cfg["model_params"]["train"] == True:
    
    print('TRAINING ABOUT TO START ... FROM ', TRAIN_START_BATCH_INDICES, 
      '  BATCH AND FOR ', cfg['train_params']['max_num_steps'], ' BATCHES', ' WITH BATCH SIZE', TRAIN_BATCH_SIZE)
    
    tr_it = iter(train_dataloader)
    progress_bar = tqdm(range(TRAIN_START_BATCH_INDICES, 
                              TRAIN_START_BATCH_INDICES + cfg["train_params"]["max_num_steps"]))
    num_iter = cfg["train_params"]["max_num_steps"]
    losses_train = []
    smooth_losses = []
    iterations = []
    metrics = []
    times = []
    lr_list = []
    model_name = cfg["model_params"]["model_name"]
    start = time.time()
    iteration = 0
    
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
        
        # Forward pass
        model.train()
        torch.set_grad_enabled(True)
        loss, _, _ = model.train_forward(data, device)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        #Gradient Value Clipping
        nn.utils.clip_grad_norm_(model.parameters(),max_norm=5.0)
        optimizer.step()
        
        # lr scheduler
        scheduler.step()
        
        losses_train.append(loss.item())
        lr_list.append(optimizer.param_groups[0]['lr'])

        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")
        if i % cfg['train_params']['checkpoint_every_n_steps'] == 0:
            sample_number = i * cfg['train_data_loader']['batch_size'] 
            
            #pred_path = f"{gettempdir()}/pred.csv"
            #model_validation_score(model, pred_path)
            
            state = {
              'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict()
            }
            torch.save(state, f'{model_name}_{sample_number}k.pth')
            iterations.append(i)
            metrics.append(np.mean(losses_train))
            times.append((time.time()-start)/60)

    sample_number = i * cfg['train_data_loader']['batch_size']
    results = pd.DataFrame({'iterations': iterations, 'metrics (avg)': metrics, 'elapsed_time (mins)': times})
    results.to_csv(f"train_metrics_{model_name}_{sample_number}k.csv", index = False)
    train_losses_csv = pd.DataFrame({'iteration': TRAIN_START_BATCH_INDICES * TRAIN_BATCH_SIZE + np.arange(len(losses_train)), 
                                 'losses_train': losses_train,
                                 'lr_list' : lr_list})
    train_losses_csv.to_csv(f"train_losses_{model_name}_{sample_number}k.csv", index = False)
    print(f"Total training time is {(time.time()-start)/60} mins")
    print(results.head())

## Validation loop

In [ ]:
if cfg["model_params"]["validate"] == True:
    pred_path = f"{gettempdir()}/pred.csv"
    model_validation_score(model, pred_path)